In [1]:
# 라이브러리 import
import pandas as pd
import time
import requests
import os
import re

import time
import json
import logging

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from concurrent.futures import ThreadPoolExecutor

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Chrome WebDriver 설정
driver = webdriver.Chrome()

# 무신사 URL 입력
driver.get("https://www.musinsa.com/category/001?gf=M&sortCode=SALE_ONE_YEAR_COUNT&color=25")

# 무한 스크롤을 통한 Product 세부 링크 수집
product_links = []  # 순서를 유지하기 위해 list 사용

try:
    scroll_increment = 500  # 스크롤 간격 (500px씩)
    while len(product_links) < 2000:  # 200개가 수집되면 종료
        # 현재 페이지에서 모든 상품 링크 수집
        products = driver.find_elements(By.CSS_SELECTOR, 'a[href^="https://www.musinsa.com/products/"]')
        
        for product in products:
            link = product.get_attribute('href')
            if link not in product_links:  # 중복 제거
                product_links.append(link)
                if len(product_links) >= 2000:  # 2000개 도달 시 종료
                    break
        
        # 스크롤을 조금씩 반복적으로 내리기
        current_scroll_position = driver.execute_script("return window.pageYOffset;")
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position + scroll_increment});")
        time.sleep(0.2)  # 짧은 대기시간 추가
        
        # 페이지 끝에 도달하면 종료
        new_scroll_position = driver.execute_script("return window.pageYOffset;")
        if current_scroll_position == new_scroll_position:
            print("더 이상 스크롤할 콘텐츠 없음")
            break

finally:
    driver.quit()

# 수집한 상품 링크 출력
print(f"총 {len(product_links)}개의 상품 링크를 수집했습니다.")
for link in product_links:
    print(link)


총 2000개의 상품 링크를 수집했습니다.
https://www.musinsa.com/products/1944612
https://www.musinsa.com/products/2723767
https://www.musinsa.com/products/2396645
https://www.musinsa.com/products/3817044
https://www.musinsa.com/products/3132887
https://www.musinsa.com/products/1582356
https://www.musinsa.com/products/4055771
https://www.musinsa.com/products/4053271
https://www.musinsa.com/products/4175458
https://www.musinsa.com/products/3640949
https://www.musinsa.com/products/1955452
https://www.musinsa.com/products/2105452
https://www.musinsa.com/products/3012570
https://www.musinsa.com/products/2082054
https://www.musinsa.com/products/1817394
https://www.musinsa.com/products/3278662
https://www.musinsa.com/products/3533479
https://www.musinsa.com/products/2403601
https://www.musinsa.com/products/3074122
https://www.musinsa.com/products/2099882
https://www.musinsa.com/products/3182421
https://www.musinsa.com/products/1375522
https://www.musinsa.com/products/4038037
https://www.musinsa.com/products/

In [ ]:
# 로그 설정
logging.basicConfig(filename='crawl_errors.log', level=logging.ERROR)

# 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI 없는 모드
chrome_options.add_argument('--disable-gpu')  # GPU 비활성화
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')  # 이미지 로딩 비활성화
chrome_options.page_load_strategy = 'eager'

driver = webdriver.Chrome(options=chrome_options)

# 이미지 저장 폴더 설정
save_folder = "C:/Users/wodnj/OneDrive/바탕 화면/BOAZ/23기 분석 BASE/미니프로젝트2/남자_다크그레이"
os.makedirs(save_folder, exist_ok=True)

# 파일 이름 지정 시 불가한 기호 제거
def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', '_', filename)

# 이미지 다운로드 함수
def download_image(url, save_path):
    try:
        img_data = requests.get(url).content
        with open(save_path, 'wb') as img_file:
            img_file.write(img_data)
        print(f"{save_path} 저장 완료")
    except Exception as e:
        logging.error(f"이미지 다운로드 오류: {e}")

all_results = dict()

# 크롤링 메인 루프
for i, link in enumerate(tqdm(product_links, desc="Processing links")):
    try:
        driver.set_page_load_timeout(30)
        driver.get(link)
        all_results[i] = dict()

        
        product_name = driver.find_element(By.CSS_SELECTOR, 
            '#root > div.sc-1f8zq2z-0.SRIds > div.sc-ysl0re-0.UluGl > div.px-4.pt-1.sc-ysl0re-1.iROunM > span').text
        sanitized_name = sanitize_filename(product_name)
        
        img_element = driver.find_element(By.CSS_SELECTOR, 'img[src*="goods_img"]')
        img_url = img_element.get_attribute('src')
        
        
        # 결과 저장
        all_results[i]['상품명'] = product_name
        all_results[i]['이미지'] = img_url
        all_results[i]['상품URL'] = link
        
        # 이미지 다운로드 병렬 처리 추가
        file_name = f"{save_folder}/{sanitized_name}.jpg"
        with ThreadPoolExecutor() as executor:
            executor.submit(download_image, img_url, file_name)
    
    except Exception as e:
        logging.error(f"오류 발생 ({link}): {e}")
        print(f"오류 발생 ({link}): {e}")

# 크롬 드라이버 종료
driver.quit()

# JSON 파일로 결과 저장
with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

print("크롤링 완료 및 결과 저장 완료!")

Processing links:   0%|                                                             | 1/2000 [00:02<1:36:03,  2.88s/it]

오류 발생 (https://www.musinsa.com/products/1944612): Message: no such element: Unable to locate element: {"method":"css selector","selector":"#root > div.sc-1f8zq2z-0.SRIds > div.sc-ysl0re-0.UluGl > div.px-4.pt-1.sc-ysl0re-1.iROunM > span"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6CC416CB5+28821]
	(No symbol) [0x00007FF6CC383840]
	(No symbol) [0x00007FF6CC22578A]
	(No symbol) [0x00007FF6CC2791BE]
	(No symbol) [0x00007FF6CC2794AC]
	(No symbol) [0x00007FF6CC2C2647]
	(No symbol) [0x00007FF6CC29F33F]
	(No symbol) [0x00007FF6CC2BF412]
	(No symbol) [0x00007FF6CC29F0A3]
	(No symbol) [0x00007FF6CC26A778]
	(No symbol) [0x00007FF6CC26B8E1]
	GetHandleVerifier [0x00007FF6CC74FCAD+3408013]
	GetHandleVerifier [0x00007FF6CC76741F+3504127]
	GetHandleVerifier [0x00007FF6CC75B5FD+3455453]
	GetHandleVerifier [0x00007FF6CC4DB

Processing links:   0%|                                                               | 2/2000 [00:03<54:01,  1.62s/it]

오류 발생 (https://www.musinsa.com/products/2723767): Message: no such element: Unable to locate element: {"method":"css selector","selector":"#root > div.sc-1f8zq2z-0.SRIds > div.sc-ysl0re-0.UluGl > div.px-4.pt-1.sc-ysl0re-1.iROunM > span"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6CC416CB5+28821]
	(No symbol) [0x00007FF6CC383840]
	(No symbol) [0x00007FF6CC22578A]
	(No symbol) [0x00007FF6CC2791BE]
	(No symbol) [0x00007FF6CC2794AC]
	(No symbol) [0x00007FF6CC2C2647]
	(No symbol) [0x00007FF6CC29F33F]
	(No symbol) [0x00007FF6CC2BF412]
	(No symbol) [0x00007FF6CC29F0A3]
	(No symbol) [0x00007FF6CC26A778]
	(No symbol) [0x00007FF6CC26B8E1]
	GetHandleVerifier [0x00007FF6CC74FCAD+3408013]
	GetHandleVerifier [0x00007FF6CC76741F+3504127]
	GetHandleVerifier [0x00007FF6CC75B5FD+3455453]
	GetHandleVerifier [0x00007FF6CC4DB

Processing links:   0%|                                                               | 3/2000 [00:04<35:46,  1.07s/it]

...

C:/Users/wodnj/OneDrive/바탕 화면/BOAZ/23기 분석 BASE/미니프로젝트2/남자_다크그레이/BIG ARCH LOGO SWEATSHIRTS [CHARCOAL].jpg 저장 완료


Processing links:  86%|█████████████████████████████████████████████████▊        | 1716/2000 [2:08:00<21:54,  4.63s/it]

C:/Users/wodnj/OneDrive/바탕 화면/BOAZ/23기 분석 BASE/미니프로젝트2/남자_다크그레이/프랑켄자수 피그먼트 오버핏맨투맨 챠콜.jpg 저장 완료


Processing links:  86%|█████████████████████████████████████████████████▊        | 1717/2000 [2:08:05<22:49,  4.84s/it]

C:/Users/wodnj/OneDrive/바탕 화면/BOAZ/23기 분석 BASE/미니프로젝트2/남자_다크그레이/유스아카이브 심볼 오버핏 롱 슬리브 - CHARCOAL.jpg 저장 완료


In [7]:
Male_Darkgray = pd.DataFrame(all_results).T

In [8]:
Male_Darkgray

,상품명,이미지,상품URL
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,오버핏 그래피티 로고 후디-멜란지그레이,https://image.msscdn.net/thumbnails/images/goo...,https://www.musinsa.com/products/2913656
...,...,...,...
995,NaN,NaN,NaN
996,NaN,NaN,NaN
997,1953 어드벤처 반팔티 애쉬그레이 FST322,https://image.msscdn.net/thumbnails/images/goo...,https://www.musinsa.com/products/3172298
998,NaN,NaN,NaN


In [ ]:
Male_Darkgray = Male_Darkgray.dropna()

In [ ]:
Male_Darkgray.to_csv('C:/Users/wodnj/OneDrive/바탕 화면/BOAZ/23기 분석 BASE/미니프로젝트2/남자_다크그레이.csv',encoding='cp949')